In [1]:
import torch
import torch.multiprocessing as mp
import torch.nn as nn
from qpth.qp import QPFunction

import numpy as np
import random

import data_generator
import params_newsvendor as params

from sklearn.preprocessing import StandardScaler

from model import VariationalLayer, VariationalNet, StandardNet, VariationalNet2

from train import TrainDecoupled

from train_normflow import TrainFlowDecoupled

import joblib

In [2]:
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda')

In [3]:
# Setting the seeds to allow replication
# Changing the seed might require hyperparameter tuning again
# Because it changes the deterministic parameters
seed_number = 2
np.random.seed(seed_number)
torch.manual_seed(seed_number)
random.seed(seed_number)

In [4]:
# Setting parameters (change if necessary)
N = 8000 # Total data size
N_train = 5000 # Training data size
N_SAMPLES = 16 # Sampling size while training
BATCH_SIZE_LOADER = 32 # Standard batch size
EPOCHS = 80 

In [5]:
nl = 1

In [6]:
# Data manipulation
N_valid = N - N_train
X, Y_original = data_generator.data_4to8(N_train, noise_level=nl)

In [7]:
# Output normalization
scaler = StandardScaler()
scaler.fit(Y_original)
tmean = torch.tensor(scaler.mean_)
tstd = torch.tensor(scaler.scale_)
joblib.dump(scaler, 'scaler_multi.gz')

['scaler_multi.gz']

In [8]:
def inverse_transform(yy):
    return yy*tstd + tmean

Y = scaler.transform(Y_original).copy()
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

In [9]:
data_train = data_generator.ArtificialDataset(X, Y)
training_loader = torch.utils.data.DataLoader(
    data_train, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=mp.cpu_count())

X_val, Y_val_original = data_generator.data_4to8(N_valid, noise_level=nl)
Y_val = scaler.transform(Y_val_original).copy()
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val_original = torch.tensor(Y_val_original, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)

data_valid = data_generator.ArtificialDataset(X_val, Y_val)
validation_loader = torch.utils.data.DataLoader(
    data_valid, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=mp.cpu_count())

input_size = X.shape[1]
output_size = Y.shape[1]

In [10]:
class SolveNewsvendorWithKKT():
    def __init__(self, params_t, n_samples):
        super(SolveNewsvendorWithKKT, self).__init__()
            
        n_items = len(params_t['c'])
        self.n_items = n_items  
        self.n_samples = n_samples
            
        # Torch parameters for KKT         
        ident = torch.eye(n_items)
        ident_samples = torch.eye(n_items*n_samples)
        ident3 = torch.eye(n_items + 2*n_items*n_samples)
        zeros_matrix = torch.zeros((n_items*n_samples, n_items*n_samples))
        zeros_array = torch.zeros(n_items*n_samples)
        ones_array = torch.ones(n_items*n_samples)
             
        self.Q = torch.diag(
            torch.hstack(
                (
                    params_t['q'], 
                    (1/n_samples)*params_t['qs'].repeat_interleave(n_samples), 
                    (1/n_samples)*params_t['qw'].repeat_interleave(n_samples)
                )
            )).to(dev)
        
        
        self.lin = torch.hstack(
                                (
                                    params_t['c'], 
                                    (1/n_samples)*params_t['cs'].repeat_interleave(n_samples), 
                                    (1/n_samples)*params_t['cw'].repeat_interleave(n_samples)
                                )).to(dev)
             
            
        shortage_ineq = torch.hstack(
            (
                -ident.repeat_interleave(n_samples, 0), 
                -ident_samples, 
                zeros_matrix
            )
        )  
        
        
        excess_ineq = torch.hstack(
            (
                ident.repeat_interleave(n_samples, 0), 
                zeros_matrix, 
                -ident_samples
            )
        )
        
        
        price_ineq = torch.hstack(
            (
                params_t['pr'], 
                zeros_array, 
                zeros_array
            )
        )
        
        
        positive_ineq = -ident3
        
        
        self.ineqs = torch.vstack(
            (
                shortage_ineq, 
                excess_ineq, 
                price_ineq, 
                positive_ineq
            )
        ).to(dev)
 
        self.uncert_bound = torch.hstack((-ones_array, ones_array)).to(dev)
        
        self.determ_bound = torch.tensor([params_t['B']]) 
        
        self.determ_bound = torch.hstack((self.determ_bound, 
                                          torch.zeros(n_items), 
                                          torch.zeros(n_items*n_samples), 
                                          torch.zeros(n_items*n_samples))).to(dev)
        
        
        
    def forward(self, y):
        """
        Applies the qpth solver for all batches and allows backpropagation.
        Formulation based on Priya L. Donti, Brandon Amos, J. Zico Kolter (2017).
        Note: The quadratic terms (Q) are used as auxiliar terms only to allow the backpropagation through the 
        qpth library from Amos and Kolter. 
        We will set them as a small percentage of the linear terms (Wilder, Ewing, Dilkina, Tambe, 2019)
        """
        
        batch_size, n_samples_items = y.size()
                
        assert self.n_samples*self.n_items == n_samples_items 

        Q = self.Q
        Q = Q.expand(batch_size, Q.size(0), Q.size(1))
        
        lin = self.lin
        lin = lin.expand(batch_size, lin.size(0))

        ineqs = torch.unsqueeze(self.ineqs, dim=0)
        ineqs = ineqs.expand(batch_size, ineqs.shape[1], ineqs.shape[2])       

        uncert_bound = (self.uncert_bound*torch.hstack((y, y)))
        determ_bound = self.determ_bound.unsqueeze(dim=0).expand(
            batch_size, self.determ_bound.shape[0])
        bound = torch.hstack((uncert_bound, determ_bound))     
        
        e = torch.DoubleTensor().to(dev)
        
        argmin = QPFunction(verbose=-1)\
            (Q.double(), lin.double(), ineqs.double(), 
             bound.double(), e, e).double()
            
        return argmin[:,:n_items]

In [11]:
cost_per_item = lambda Z, Y : params_t['q'].to(dev)*Z.to(dev)**2 \
                            + params_t['qs'].to(dev)*(torch.max(torch.zeros((n_items)).to(dev),Y.to(dev)-Z.to(dev)))**2 \
                            + params_t['qw'].to(dev)*(torch.max(torch.zeros((n_items)).to(dev),Z.to(dev)-Y.to(dev)))**2 \
                            + params_t['c'].to(dev)*Z.to(dev) \
                            + params_t['cs'].to(dev)*torch.max(torch.zeros((n_items)).to(dev),Y.to(dev)-Z.to(dev)) \
                            + params_t['cw'].to(dev)*torch.max(torch.zeros((n_items)).to(dev),Z.to(dev)-Y.to(dev))


def reshape_outcomes(y_pred):
    n_samples = y_pred.shape[0]
    batch_size = y_pred.shape[1]
    n_items = y_pred.shape[2]

    y_pred = y_pred.permute((1, 2, 0)).reshape((batch_size, n_samples*n_items))

    return y_pred

def calc_f_por_item(y_pred, y):
    y_pred = reshape_outcomes(y_pred)
    z_star =  argmin_solver(y_pred)
    f_per_item = cost_per_item(z_star, y)
    return f_per_item

def calc_f_per_day(y_pred, y):
    f_per_item = calc_f_por_item(y_pred, y)
    f = torch.sum(f_per_item, 1)
    return f

def cost_fn(y_pred, y):
    f = calc_f_per_day(y_pred, y)
    f_total = torch.mean(f)
    return f_total

In [12]:
h_ann = StandardNet(input_size, output_size, 0).to(dev)
h_bnn = VariationalNet(N_SAMPLES, input_size, output_size, 1.0).to(dev)

opt_h_ann = torch.optim.Adam(h_ann.parameters(), lr=0.0010)
opt_h_bnn = torch.optim.Adam(h_bnn.parameters(), lr=0.0015)

mse_loss = nn.MSELoss(reduction='none')

In [13]:
train_ANN = TrainDecoupled(
                    bnn = False,
                    model=h_ann,
                    opt=opt_h_ann,
                    loss_data=mse_loss,
                    K=0.0,
                    training_loader=training_loader,
                    validation_loader=validation_loader
                )

train_ANN.train(EPOCHS=120)
model_ann = train_ANN.model

------------------EPOCH 1------------------
DATA LOSS 	 train 0.496 valid 0.337
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.5 valid 0.34
------------------EPOCH 2------------------
DATA LOSS 	 train 0.306 valid 0.283
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.31 valid 0.28
------------------EPOCH 3------------------
DATA LOSS 	 train 0.26 valid 0.261
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.26 valid 0.26
------------------EPOCH 4------------------
DATA LOSS 	 train 0.237 valid 0.247
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.24 valid 0.25
------------------EPOCH 5------------------
DATA LOSS 	 train 0.224 valid 0.236
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.22 valid 0.24
------------------EPOCH 6------------------
DATA LOSS 	 train 0.215 valid 0.226
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.21 valid 0.23
------------------EPOCH 7------------------
DATA LOSS 	 train 0.206 valid 0.217
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.21 valid

DATA LOSS 	 train 0.063 valid 0.093
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.06 valid 0.09
------------------EPOCH 59------------------
DATA LOSS 	 train 0.062 valid 0.093
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.06 valid 0.09
------------------EPOCH 60------------------
DATA LOSS 	 train 0.062 valid 0.092
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.06 valid 0.09
------------------EPOCH 61------------------
DATA LOSS 	 train 0.061 valid 0.092
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.06 valid 0.09
------------------EPOCH 62------------------
DATA LOSS 	 train 0.06 valid 0.092
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.06 valid 0.09
------------------EPOCH 63------------------
DATA LOSS 	 train 0.06 valid 0.092
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.06 valid 0.09
------------------EPOCH 64------------------
DATA LOSS 	 train 0.059 valid 0.091
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.06 valid 0.09
------------------EPOCH 65------

DATA LOSS 	 train 0.05 valid 0.09
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 116------------------
DATA LOSS 	 train 0.05 valid 0.089
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 117------------------
DATA LOSS 	 train 0.05 valid 0.089
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 118------------------
DATA LOSS 	 train 0.049 valid 0.09
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 119------------------
DATA LOSS 	 train 0.049 valid 0.091
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 120------------------
DATA LOSS 	 train 0.049 valid 0.089
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09


In [14]:
train_BNN = TrainDecoupled(
                    bnn = True,
                    model=h_bnn,
                    opt=opt_h_bnn,
                    loss_data=mse_loss,
                    K=1.0,
                    training_loader=training_loader,
                    validation_loader=validation_loader
                )

train_BNN.train(EPOCHS=150)
model_bnn = train_BNN.model

------------------EPOCH 1------------------
DATA LOSS 	 train 0.893 valid 0.628
KL LOSS 	 train 3.31 valid 3.31
ELBO LOSS 	 train 4.21 valid 3.94
------------------EPOCH 2------------------
DATA LOSS 	 train 0.546 valid 0.481
KL LOSS 	 train 3.32 valid 3.33
ELBO LOSS 	 train 3.87 valid 3.81
------------------EPOCH 3------------------
DATA LOSS 	 train 0.435 valid 0.404
KL LOSS 	 train 3.33 valid 3.35
ELBO LOSS 	 train 3.77 valid 3.75
------------------EPOCH 4------------------
DATA LOSS 	 train 0.372 valid 0.357
KL LOSS 	 train 3.35 valid 3.36
ELBO LOSS 	 train 3.72 valid 3.72
------------------EPOCH 5------------------
DATA LOSS 	 train 0.333 valid 0.326
KL LOSS 	 train 3.37 valid 3.38
ELBO LOSS 	 train 3.7 valid 3.7
------------------EPOCH 6------------------
DATA LOSS 	 train 0.304 valid 0.303
KL LOSS 	 train 3.39 valid 3.4
ELBO LOSS 	 train 3.69 valid 3.7
------------------EPOCH 7------------------
DATA LOSS 	 train 0.283 valid 0.285
KL LOSS 	 train 3.4 valid 3.41
ELBO LOSS 	 train

DATA LOSS 	 train 0.093 valid 0.126
KL LOSS 	 train 3.99 valid 3.99
ELBO LOSS 	 train 4.09 valid 4.12
------------------EPOCH 58------------------
DATA LOSS 	 train 0.093 valid 0.125
KL LOSS 	 train 4.0 valid 4.0
ELBO LOSS 	 train 4.09 valid 4.13
------------------EPOCH 59------------------
DATA LOSS 	 train 0.092 valid 0.125
KL LOSS 	 train 4.01 valid 4.01
ELBO LOSS 	 train 4.1 valid 4.14
------------------EPOCH 60------------------
DATA LOSS 	 train 0.092 valid 0.125
KL LOSS 	 train 4.01 valid 4.02
ELBO LOSS 	 train 4.11 valid 4.14
------------------EPOCH 61------------------
DATA LOSS 	 train 0.091 valid 0.125
KL LOSS 	 train 4.02 valid 4.02
ELBO LOSS 	 train 4.11 valid 4.15
------------------EPOCH 62------------------
DATA LOSS 	 train 0.09 valid 0.124
KL LOSS 	 train 4.03 valid 4.03
ELBO LOSS 	 train 4.12 valid 4.16
------------------EPOCH 63------------------
DATA LOSS 	 train 0.089 valid 0.123
KL LOSS 	 train 4.03 valid 4.04
ELBO LOSS 	 train 4.12 valid 4.16
------------------EP

DATA LOSS 	 train 0.075 valid 0.114
KL LOSS 	 train 4.18 valid 4.18
ELBO LOSS 	 train 4.25 valid 4.29
------------------EPOCH 114------------------
DATA LOSS 	 train 0.075 valid 0.113
KL LOSS 	 train 4.18 valid 4.19
ELBO LOSS 	 train 4.25 valid 4.3
------------------EPOCH 115------------------
DATA LOSS 	 train 0.075 valid 0.113
KL LOSS 	 train 4.18 valid 4.18
ELBO LOSS 	 train 4.26 valid 4.3
------------------EPOCH 116------------------
DATA LOSS 	 train 0.075 valid 0.113
KL LOSS 	 train 4.18 valid 4.18
ELBO LOSS 	 train 4.26 valid 4.3
------------------EPOCH 117------------------
DATA LOSS 	 train 0.074 valid 0.113
KL LOSS 	 train 4.18 valid 4.18
ELBO LOSS 	 train 4.26 valid 4.3
------------------EPOCH 118------------------
DATA LOSS 	 train 0.074 valid 0.113
KL LOSS 	 train 4.18 valid 4.18
ELBO LOSS 	 train 4.26 valid 4.3
------------------EPOCH 119------------------
DATA LOSS 	 train 0.074 valid 0.113
KL LOSS 	 train 4.18 valid 4.18
ELBO LOSS 	 train 4.26 valid 4.29
---------------

In [15]:
train_flow = TrainFlowDecoupled(steps=2500, input_size=4, output_size=8)
model_flow = train_flow.train(X, Y, X_val, Y_val)

step: 0, train loss: 5.79808, val loss: 5.70293
step: 50, train loss: 4.07737, val loss: 4.09179
step: 100, train loss: 3.27692, val loss: 3.3427
step: 150, train loss: 2.8138, val loss: 2.90492
step: 200, train loss: 2.49744, val loss: 2.60201
step: 250, train loss: 2.2451, val loss: 2.3652
step: 300, train loss: 2.04975, val loss: 2.17902
step: 350, train loss: 1.91018, val loss: 2.04559
step: 400, train loss: 1.79934, val loss: 1.95387
step: 450, train loss: 1.74476, val loss: 1.90066
step: 500, train loss: 1.67851, val loss: 1.85055
step: 550, train loss: 1.62338, val loss: 1.81214
step: 600, train loss: 1.59577, val loss: 1.7853
step: 650, train loss: 1.57174, val loss: 1.77225
step: 700, train loss: 1.5458, val loss: 1.75836
step: 750, train loss: 1.52965, val loss: 1.74595
step: 800, train loss: 1.5259, val loss: 1.73437
step: 850, train loss: 1.51237, val loss: 1.73382
step: 900, train loss: 1.48909, val loss: 1.7271
step: 950, train loss: 1.51302, val loss: 1.72441
step: 1000,

In [16]:
n_items = output_size

In [17]:
params_t, _ = params.get_params(n_items, seed_number)

In [18]:
# Propagating predictions to Newsvendor Problem
M = 16

Y_pred_ANN = train_ANN.model(X_val).unsqueeze(0)
Y_pred_ANN = inverse_transform(Y_pred_ANN)

train_BNN.model.update_n_samples(n_samples=M)
Y_pred_BNN = train_BNN.model.forward_dist(X_val)
Y_pred_BNN = inverse_transform(Y_pred_BNN)
#M = Y_pred_BNN.shape[0]

N = X_val.shape[0]
Y_pred_flow = torch.zeros((M, N, n_items))
for i in range(0, N):
    Y_pred_flow[:,i,:] = model_flow.condition(X_val[i]).sample(torch.Size([M,])).squeeze()
Y_pred_flow = inverse_transform(Y_pred_flow)

In [20]:
mse_loss = nn.MSELoss()
print(mse_loss(Y_pred_ANN.mean(axis=0), Y_val_original))
print(mse_loss(Y_pred_BNN.mean(axis=0), Y_val_original))
print(mse_loss(Y_pred_flow.mean(axis=0), Y_val_original))

tensor(1.8488, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(2.3078, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(2.3018, dtype=torch.float64)


In [21]:
params_t['B'] = torch.tensor([20000])

In [22]:
params_t['cs'] = 5*params_t['cs']

In [23]:
# Construct the solver
newsvendor_solve_kkt = SolveNewsvendorWithKKT(params_t, 1)
newsvendor_solve_kkt_M = SolveNewsvendorWithKKT(params_t, M)

In [24]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_ANN.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_ANN.shape[1]
    f_total += cost_fn(Y_pred_ANN[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

/home/alan/Desktop/envs/pao_env/lib/python3.8/site-packages/qpth/qp.py:83: UserWarning: torch.eig is deprecated in favor of torch.linalg.eig and will be removed in a future PyTorch release.
torch.linalg.eig returns complex tensors of dtype cfloat or cdouble rather than real tensors mimicking complex tensors.
L, _ = torch.eig(A)
should be replaced with
L_complex = torch.linalg.eigvals(A)
and
L, V = torch.eig(A, eigenvectors=True)
should be replaced with
L_complex, V_complex = torch.linalg.eig(A) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2910.)
  e, _ = torch.eig(Q[i])


0 tensor(219.7427, dtype=torch.float64, grad_fn=<AddBackward0>)
1 tensor(431.2005, dtype=torch.float64, grad_fn=<AddBackward0>)
2 tensor(663.9111, dtype=torch.float64, grad_fn=<AddBackward0>)
3 tensor(907.9290, dtype=torch.float64, grad_fn=<AddBackward0>)
4 tensor(1187.4045, dtype=torch.float64, grad_fn=<AddBackward0>)
5 tensor(1443.8464, dtype=torch.float64, grad_fn=<AddBackward0>)
6 tensor(1676.6126, dtype=torch.float64, grad_fn=<AddBackward0>)
7 tensor(1896.4263, dtype=torch.float64, grad_fn=<AddBackward0>)
8 tensor(2134.6857, dtype=torch.float64, grad_fn=<AddBackward0>)
9 tensor(2359.6012, dtype=torch.float64, grad_fn=<AddBackward0>)
10 tensor(2640.7107, dtype=torch.float64, grad_fn=<AddBackward0>)
11 tensor(2843.3051, dtype=torch.float64, grad_fn=<AddBackward0>)
12 tensor(3059.9432, dtype=torch.float64, grad_fn=<AddBackward0>)
13 tensor(3324.4056, dtype=torch.float64, grad_fn=<AddBackward0>)
14 tensor(3535.1858, dtype=torch.float64, grad_fn=<AddBackward0>)
15 tensor(3771.9072, dty

In [25]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt_M.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_BNN.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_BNN.shape[1]
    f_total += cost_fn(Y_pred_BNN[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

0 tensor(205.9630, dtype=torch.float64, grad_fn=<AddBackward0>)
1 tensor(424.5264, dtype=torch.float64, grad_fn=<AddBackward0>)
2 tensor(685.7454, dtype=torch.float64, grad_fn=<AddBackward0>)
3 tensor(883.9132, dtype=torch.float64, grad_fn=<AddBackward0>)
4 tensor(1136.4299, dtype=torch.float64, grad_fn=<AddBackward0>)
5 tensor(1384.5668, dtype=torch.float64, grad_fn=<AddBackward0>)
6 tensor(1635.8922, dtype=torch.float64, grad_fn=<AddBackward0>)
7 tensor(1844.2177, dtype=torch.float64, grad_fn=<AddBackward0>)
8 tensor(2082.1145, dtype=torch.float64, grad_fn=<AddBackward0>)
9 tensor(2298.9089, dtype=torch.float64, grad_fn=<AddBackward0>)
10 tensor(2548.6087, dtype=torch.float64, grad_fn=<AddBackward0>)
11 tensor(2745.0889, dtype=torch.float64, grad_fn=<AddBackward0>)
12 tensor(2936.1108, dtype=torch.float64, grad_fn=<AddBackward0>)
13 tensor(3176.2841, dtype=torch.float64, grad_fn=<AddBackward0>)
14 tensor(3363.1397, dtype=torch.float64, grad_fn=<AddBackward0>)
15 tensor(3568.9273, dty

AttributeError: 'NoneType' object has no attribute 'double'

In [ ]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt_M.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_flow.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_flow.shape[1]
    f_total += cost_fn(Y_pred_flow[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

0 tensor(190.3964, dtype=torch.float64)
1 tensor(369.9579, dtype=torch.float64)
2 tensor(590.4687, dtype=torch.float64)
3 tensor(782.5824, dtype=torch.float64)
4 tensor(975.4891, dtype=torch.float64)
5 tensor(1171.7013, dtype=torch.float64)
6 tensor(1360.3300, dtype=torch.float64)
7 tensor(1517.9134, dtype=torch.float64)
8 tensor(1717.5365, dtype=torch.float64)
9 tensor(1877.5952, dtype=torch.float64)


In [24]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_val_original.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_val_original.shape[1]
    f_total += cost_fn(Y_val_original[i_low:i_up,:].unsqueeze(0), Y_val_original[i_low:i_up,:])/n_batches
    print(f_total)

tensor(13554.7681, dtype=torch.float64)


In [27]:
Y_val_original

tensor([[ 3.0070,  5.6180,  0.4540,  ...,  5.6970,  2.0970,  4.0900],
        [ 8.1370,  4.5640,  6.4230,  ..., 10.3960, 19.9330, 21.6580],
        [ 0.3950,  7.5780,  4.4040,  ..., 11.2520, 15.8290, 11.5620],
        ...,
        [ 5.4410,  3.4550,  1.9840,  ...,  2.0790,  9.3480, 15.2870],
        [ 5.2250,  1.4480,  3.2950,  ...,  4.7980,  9.5330, 11.0260],
        [ 3.1210,  2.5350,  2.0630,  ...,  3.9480, 14.1670, 15.3820]])

In [33]:
Y_pred_BNN[:,:10,:].shape

torch.Size([16, 10, 8])

In [39]:
params_t

{'q': tensor([3., 6., 5., 7., 7., 4., 6., 5.]),
 'qs': tensor([23., 20., 18., 21., 17., 23., 15., 24.]),
 'qw': tensor([16., 14., 17., 13., 14., 17., 10., 16.]),
 'c': tensor([27., 29., 31., 34., 26., 26., 27., 30.]),
 'cs': tensor([265., 227., 223., 221., 229., 212., 198., 260.]),
 'cw': tensor([ 87., 119.,  83.,  82.,  88.,  89.,  92., 102.]),
 'pr': tensor([247., 283., 315., 326., 291., 312., 275., 275.]),
 'B': tensor([7271.6777]),
 'si': tensor([262., 220., 334., 298., 256., 345., 305., 224.]),
 'S': tensor([9181.8213])}

In [38]:
Y_val_original[:3,:]

tensor([[ 3.0070,  5.6180,  0.4540,  1.4640,  6.9450,  5.6970,  2.0970,  4.0900],
        [ 8.1370,  4.5640,  6.4230, 15.1250, 12.7400, 10.3960, 19.9330, 21.6580],
        [ 0.3950,  7.5780,  4.4040, 10.9700,  7.7020, 11.2520, 15.8290, 11.5620]])

In [67]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

(params_t['pr']*argmin_solver(reshape_outcomes(Y_val_original[:3,:].unsqueeze(0)))).sum(axis=1)

tensor([ 8452.7811, 20000.0000, 20000.0000], dtype=torch.float64)

In [37]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

argmin_solver(reshape_outcomes(Y_val_original[:3,:].unsqueeze(0)))

tensor([[3.0070e+00, 5.2556e+00, 2.9634e-01, 6.2984e-01, 5.9350e+00, 4.6487e+00,
         1.6025e+00, 4.0900e+00],
        [3.7207e+00, 6.8705e-16, 6.1038e-17, 2.5419e+00, 1.3610e+00, 3.8036e-01,
         4.9692e+00, 1.3247e+01],
        [1.5357e-13, 2.2781e+00, 1.3106e-16, 2.9616e+00, 1.4749e+00, 4.6191e+00,
         6.0149e+00, 7.7712e+00]], dtype=torch.float64)

In [41]:
Y_pred_BNN[:,:3,:].mean(axis=0)

tensor([[ 3.3046,  5.8885,  2.6716,  2.5851,  9.3277,  6.4692,  3.1727,  4.8759],
        [ 7.8597,  3.6131,  7.0088, 15.8783, 11.6528,  8.6631, 20.9295, 22.7702],
        [ 2.2485,  4.9931,  7.3643, 12.8174,  6.8757, 10.4046, 17.9280, 13.7108]],
       dtype=torch.float64, grad_fn=<MeanBackward1>)

In [35]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt_M.forward(y_pred)
    return z_star

argmin_solver(reshape_outcomes(Y_pred_BNN[:,:3,:]))

tensor([[ 3.0621e+00,  4.5241e+00,  8.4962e-01,  4.6565e-01,  6.5761e+00,
          4.3091e+00,  1.2404e+00,  4.4232e+00],
        [ 3.3379e+00,  1.6379e-17, -4.7237e-15,  2.9384e+00,  4.1553e-01,
         -3.1687e-15,  5.4916e+00,  1.4030e+01],
        [ 1.0761e+00,  3.4230e-16,  1.0293e-15,  3.9541e+00,  4.8021e-01,
          3.5071e+00,  7.0721e+00,  9.2294e+00]], dtype=torch.float64,
       grad_fn=<SliceBackward0>)